In [3]:
def remove_null_columns(df):
    mask     = df.isnull().all()
    null     = df.columns[mask]
    not_null = df.columns[np.logical_not(mask)]

    print(f'{len(null)+len(not_null)} colunas \n{len(not_null)} com pelo menos uma informacao \n{len(null)} sem nenhuma informacao')
    display(pd.Series(null))
    
    final_cols = not_null.tolist() + null.tolist()
    
    
    df = df[final_cols]

    return df

def organizer(df,cols):
    
    not_in = [col for col in df.columns.tolist() if col not in cols]
    final_cols = cols + not_in
    df = df[final_cols]
    
    return df

In [2]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os


import requests
import urllib

## Source
 - [Relação de Contratações e Compras / Lei 13.979/2020](https://www.saopaulo.sp.gov.br/coronavirus/transparencia/)


In [4]:
valores = pd.read_csv('../data/orcamento/sp/COVID.csv',sep=';', encoding='iso-8859-9')

#Colunas que tem no portal mas n tem na tabela:  ['Fonte','Local de Entrega']
### Link Nota de Empenho
#http://siafem-appws.fazenda.sp.gov.br/ServicesSiafem/NE_PDF?UG=080358&GESTAO=00001&NE=2020NE00119
#url + Código UG Documento + GESTAO? + Número Empenho



# cols = ['Secretaria','Número Processo','Descrição','Nome Credor','CNPJ CPF','Descrição Processo',
#         'Finalidade', 'Quantidade Item','Valor Unitário', 'Valor NE','Data Emissão',
#         'Tipo Documento','Número Documento','Valor NP','Valor de Empenho no Processo']

# valores = organizer(valores,cols)

In [8]:
valores.head()

,Nome Órgão,Número Processo,Descrição Processo,Modalidade de Contratação,Empresa Contratada,CNPJ CPF,Descrição Processo.1,Item / Finalidade,Quantidade Item,Embalagem,Valor Unitário,Valor Total,Número Empenho,Data Emissão,Tipo Licitação,Materiais/Serviços,Número Oferta Compra,Local Entrega,Nome Gestão
0,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"SABONETE,LIQUIDO PEROLADO, ESSENCIA ERVA DOCE,PH ENTRE 6,5 - 7,0",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,27,246 - GALAO 5 L,"22,3","602,1",2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
1,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"HIGIENIZADOR, 70% ALCOOL ETILICO",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,170,797 - FRASCO 1000 ML,"18,738","3185,46",2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
2,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"PAPEL TOALHA, 22,5 X 20 CM",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,180,104 - PACOTE 1000 FL,"11,6",2088,2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
3,SECRETARIA DA EDUCACAO,20318/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"HIGIENIZADOR, 70% ALCOOL ETILICO",AQUISIçãO DE KIT DE HIGIENE PRA COMBATE AO COVID-19 PARA E.E. PARQUE JARDIM HELENA,15,797 - FRASCO 1000 ML,"18,738","281,07",2020NE00268,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
4,SECRETARIA DA EDUCACAO,20318/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"SABONETE,LIQUIDO PEROLADO, ESSENCIA ERVA DOCE,PH ENTRE 6,5 - 7,0",AQUISIçãO DE KIT DE HIGIENE PRA COMBATE AO COVID-19 PARA E.E. PARQUE JARDIM HELENA,3,246 - GALAO 5 L,"22,8","68,4",2020NE00268,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO


In [4]:
processos = pd.read_csv('../data/orcamento/sp/BaseDados_SPSemPapel.csv',sep=';', encoding='iso-8859-9')

#Colunas que tem no portal mas n tem na tabela:[]
#link processo https://www.saopaulo.sp.gov.br/guia-coronavirus/assets/docs-transparencia/PCSPPRC202000428V01.pdf
#url + Número do Processo + V01.pdf

cols = ['Órgão','Número do Processo','Tipo de Processo','Descrição']

processos = organizer(processos,cols)

In [5]:
contratos = pd.read_csv('../data/orcamento/sp/BaseDados_IMESP.csv',sep=';', encoding='iso-8859-9')

contratos = remove_null_columns(contratos)


# #Colunas que tem no portal mas n tem na tabela:[]
# cols = ['secretaria','processo','modalidadedescricao','contratado','dataAssinatura',
#         'dataPublicacao1','prazocontrato','tipoprazo','statusLicitacao','licitacaoLink']

# contratos = organizer(contratos,cols)

10 colunas 
10 com pelo menos uma informacao 
0 sem nenhuma informacao


Series([], dtype: object)

In [6]:
valores.head()

,Nome Órgão,Número Processo,Descrição Processo,Modalidade de Contratação,Empresa Contratada,CNPJ CPF,Descrição Processo.1,Item / Finalidade,Quantidade Item,Embalagem,Valor Unitário,Valor Total,Número Empenho,Data Emissão,Tipo Licitação,Materiais/Serviços,Número Oferta Compra,Local Entrega,Nome Gestão
0,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"SABONETE,LIQUIDO PEROLADO, ESSENCIA ERVA DOCE,PH ENTRE 6,5 - 7,0",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,27,246 - GALAO 5 L,"22,3","602,1",2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
1,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"HIGIENIZADOR, 70% ALCOOL ETILICO",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,170,797 - FRASCO 1000 ML,"18,738","3185,46",2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
2,SECRETARIA DA EDUCACAO,20309/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"PAPEL TOALHA, 22,5 X 20 CM",AQUISIçãO DE KIT DE HIGIENE PARA COMBATE AO COVID-19 PARA A E.E. BRENO DI GRADO,180,104 - PACOTE 1000 FL,"11,6",2088,2020NE00269,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
3,SECRETARIA DA EDUCACAO,20318/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"HIGIENIZADOR, 70% ALCOOL ETILICO",AQUISIçãO DE KIT DE HIGIENE PRA COMBATE AO COVID-19 PARA E.E. PARQUE JARDIM HELENA,15,797 - FRASCO 1000 ML,"18,738","281,07",2020NE00268,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO
4,SECRETARIA DA EDUCACAO,20318/20,KIT DE HIGIENE,DISPENSA DE LICITACAO,PEDRO AUGUSTO DA CRUZ - EMPORIO ME,27695599000181,"SABONETE,LIQUIDO PEROLADO, ESSENCIA ERVA DOCE,PH ENTRE 6,5 - 7,0",AQUISIçãO DE KIT DE HIGIENE PRA COMBATE AO COVID-19 PARA E.E. PARQUE JARDIM HELENA,3,246 - GALAO 5 L,"22,8","68,4",2020NE00268,43957,5,Materiais,NaN,RUA MOHAMAD I SALEH 979,GOVERNO DO ESTADO DE SAO PAULO


## CNPJS

In [15]:
df = valores.copy()

df['id'] = df['CNPJ CPF'].astype(str).str.replace('.','').str.replace('-','').str.replace('/','')
df['descricao'] =   df['Descrição Processo.1'] + \
                    '\n Quantidade Item: '     +  df['Quantidade Item'].astype(str) + \
                    '\n Valor Unitário: '      +  df['Valor Unitário'].astype(str)  + \
                    '\n Valor Total: '         +  df['Valor Total'].astype(str)     + \
                    '\n Local Entrega: '       +  df['Local Entrega'].astype(str)   + \
                    '\n Número Processo: '     +  df['Número Processo'].astype(str)





df['Valor Total'] = df['Valor Total'].str.replace('.','').str.replace(',','.').astype(float)
df['descricao']   = df['descricao'].astype(str)



df_values = df.groupby(by=['id','CNPJ CPF','Empresa Contratada'], as_index=False).agg({'descricao':'\n\n'.join, 'Valor Total':sum})
df_values = df_values.rename(columns={'CNPJ CPF':'CNPJ'})

In [8]:
url = f'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'

df_final = pd.DataFrame()
for cnpj in df['id'].unique():
    url_csv = url + f'{cnpj}.csv'

    
    try:
        dd = pd.read_csv(url_csv)
        print(url_csv)
        dd = dd.drop(columns= ['Id','CNPJ','Unidade Cadastradora','Ramo do Negócio','Porte da Empresa','CNAE Secundário','Caixa Postal','Recadastrado'])
        dd['id'] = cnpj
        
    except:
        dd = pd.DataFrame([cnpj],columns=['id'])
        
        print(cnpj, 'nao existe')

    
    df_final = pd.concat([df_final,dd],axis=0)

df_final = df_final.sort_values(by='Razão Social')


27695599000181 nao existe
3122375000112 nao existe
17829173000110 nao existe
34346501000146 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/43085349000186.csv
53917324000155 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/14709296000129.csv
5268903000171 nao existe
5788117000103 nao existe
6127890000183 nao existe
9251627000190 nao existe
2881877000164 nao existe
1449930000602 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/54178983000180.csv
5411043000183 nao existe
7439329000364 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/27608037000153.csv
2563570000115 nao existe
331788002324 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/10368534000129.csv
6105362000123 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/45985371000108.csv
8304991000108 nao existe
2535707000128 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/14365637000196.csv
9

In [18]:
ddd = pd.merge(df_values, df_final, on='id', how='outer').sort_values(by='Razão Social').drop(columns=['id'])

In [19]:
ddd.to_excel('../data/orcamento/sp/lista_cnpjs.xlsx', index=False)

## Cidades

In [4]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os

## Fonte
 - [Boletim](https://www.sjc.sp.gov.br/servicos/saude/coronavirus/atos-e-despesas/)

In [5]:
#sjc

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

In [6]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [10]:
cols = ['material','descricao','quantidade_af','unidade','valor_unitario','valor_total','numero_af','data','modalidade_de_compra','razao_social','unidade_orcamentaria','codigo_dotacao_orcamentaria']
df.columns = cols
df = df[df['material']!='MATERIAL']

df['valor_unitario'] = df['valor_unitario'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))
df['valor_total'] = df['valor_total'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))

df = df.replace('\n','')

In [13]:
df.to_csv(f'../data/orcamento/csv/sjc/{file_name}.csv', index=False)

## Fonte
 - [Boletim](http://www.osasco.sp.gov.br/portal-da-transparencia)

In [ ]:
#osasco

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

## Taubate

## Fonte
 - [Boletim](http://leideacesso.etransparencia.com.br/taubate.prefeitura.sp/TDAPortalClient.aspx?414)

In [33]:
df = pd.read_excel('../data/orcamento/csv/tbt/contratacoes_taubate.xlsx')

df['cnpj'] = df['Fornecedor'].apply(lambda x: x.split('(')[1].split(')')[0])
df['id'] = df['cnpj'].str.replace('.','').str.replace('-','').str.replace('/','')



df['Finalidade'] = df['Finalidade'] + '. Valor: ' +  df['Despesa Contratada (Empenhada)'].astype(str)
df_values = df.groupby(by=['id','cnpj','Fornecedor'], as_index=False).agg({'Finalidade':'\n\n'.join, 'Despesa Contratada (Empenhada)':sum})
df_values = df_values.rename(columns={'cnpj':'CNPJ','Despesa Contratada (Empenhada)':'valor total'})


In [35]:
url = f'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'

df_final = pd.DataFrame()
for cnpj in df['id'].unique():
    url_csv = url + f'{cnpj}.csv'

    
    try:
        dd = pd.read_csv(url_csv)
        print(url_csv)
        dd = dd.drop(columns= ['Id','CNPJ','Unidade Cadastradora','Ramo do Negócio','Porte da Empresa','CNAE Secundário','Caixa Postal','Recadastrado'])
        dd['id'] = cnpj
        
    except:
        dd = pd.DataFrame([cnpj],columns=['id'])
        
        print(cnpj, 'nao existe')

    
    df_final = pd.concat([df_final,dd],axis=0)

df_final = df_final.sort_values(by='Razão Social')


http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/61610283000188.csv
03961840000109 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/61699567000192.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/02190877000118.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/08093976000168.csv
26864805000177 nao existe
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/67729178000491.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04063331000121.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/05515873000150.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/55309074000104.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/52202744000192.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03945035000191.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/07118264000193.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/14112327000160.csv
157254890001

In [36]:
ddd = pd.merge(df_values, df_final, on='id', how='outer').sort_values(by='Razão Social').drop(columns=['id'])

In [38]:
ddd.to_excel('../data/orcamento/csv/tbt/lista_cnpjs_tbt.xlsx', index=False)